In [12]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import qmc

c = 3e8 # Speed of Light

class Muon:
    def __init__(self, x, y, z, v, v_theta, v_phi):
        self.v_theta = v_theta
        self.v_phi = v_phi
        self.z_list = [z]
        self.v = v
        self.x_list = [x]
        self.y_list = [y]
        self.lifetime = 3

    def propagate(self, time_step):
        time = 0
        while time < self.lifetime:
            self.x_list.append(self.v * time_step * np.sin(self.phi) * np.cos(self.theta))
            self.y_list.append(self.v * time_step * np.sin(self.phi) * np.sin(self.theta))
            self.z_list.append(self.v * time_step * np.cos(self.phi))
            time += time_step

# Defines 3D regions where scintillation occurs.
class Scintillator:
    def __init__(self, r, z):
        self.r = r
        self.z = z

    def has_muon(self, muon):
        for i in range(muon.z_list):
            if self.z[0] <= muon.z_list[i] <= self.z[1] and np.sqrt(muon.x_list**2 + muon.y_list**2) <= self.r:
                return True
        return False

experiment = [Scintillator(1, [10,9]), Scintillator(1, [8,0])]

In [47]:
# returns random velocities to be used to build a muon class
def random_velocities(N):
    sampler = qmc.LatinHypercube(d = 1)
    sample = sampler.random(n = N)
    velocities = qmc.scale(sample,  0.9990, 0.9995)
    velocities *= c
    return velocities

def random_theta(N):
    sampler = qmc.LatinHypercube(d = 1)
    sample = sampler.random(n = N)
    thetas = qmc.scale(sample,  0, 2*np.pi)
    return thetas

def random_phi(N):
    sampler = qmc.LatinHypercube(d = 1)
    sample = sampler.random(n = N)
    phis = qmc.scale(sample,  0, np.pi)
    return phis

def random_r(N):
    sampler = qmc.LatinHypercube(d = 1)
    sample = sampler.random(n = N)
    return sample

def random_muons(N):
    rand_r = random_r(N)
    rand_theta = random_theta(N)
    rand_x = rand_r * np.sin(rand_theta)
    rand_y = rand_r * np.cos(rand_theta)
    rand_v = random_velocities(N)
    rand_v_theta = random_theta(N)
    rand_v_phi = random_phi(N)
    muon_list = []
    for i in [0, N-1]:
        muon_list.append(Muon(rand_x[i], rand_y[i], 1, rand_v[i], rand_v_theta[i], rand_v_phi[i]))
    return muon_list

[<__main__.Muon at 0x2029d728670>, <__main__.Muon at 0x2029d7284c0>]